In [3]:
import json 
from openai import OpenAI
from dotenv import load_dotenv
import os
import tiktoken

In [5]:
path_data = os.path.join("Data")

In [7]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [9]:
openai = OpenAI()

In [11]:
with open(f"{path_data}\\dados.json", encoding="utf-8") as file:
    dados = json.load(file)

In [21]:
start_prompt = """
Seu objetivo é recomendar NFRs para projetos que ainda não possuem NFRs associados.
Utilize como critério os campos: 'Descritivo_da_US', 'Modulo', 'Operacao',
'Camada', 'Linguagem', 'Tarefa_mapeada' e 'Tarefa_original'.

forneça uma recomendação para os campos que estão na expectativa de resposta de 
acordo com os dados informados do novo projeto.

Formato esperado da resposta:
[
  {
    "Tarefa_ID": "TAxx",
    "NFR_Tipo": "Reliability",
    "NFR_Atributo": "availability",
    "NFR_Sentença": "The system must notify users when the function is unavailable."
  },
  ...
]

A recomendação deve ser feita com base nesse novo projeto:
[
    {
        "fr": "Como administrador, desejo editar permissões de acesso a relatórios financeiros.",
        "dominio": "Office/Business",
        "plataforma": "Web",
        "arquitetura": "Client-server",
        "modulo": "Autenticação",
        "operacao": "Editar_permissão",
        "tecnologias": {"Angular", "TypeScript", "HTML", "CSS"}
    }
]
Aqui estão os dados:
"""

In [15]:
'''
Contém toda a base de dados dos projetos
'''

json_data = json.dumps(dados, indent=2, ensure_ascii=False)

In [23]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Você é um especialista em engenharia de requisitos"},
        {"role": "user", "content": start_prompt}
        ]
    )
print("\n--- RESPOSTA ---\n")
print(response.choices[0].message.content)


--- RESPOSTA ---

Com base nas informações do novo projeto, aqui está uma recomendação de NFRs (Non-Functional Requirements) que podem ser associados:

```json
[
  {
    "Tarefa_ID": "TA01",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "authentication",
    "NFR_Sentença": "The system must ensure that only authenticated users can edit access permissions for financial reports."
  },
  {
    "Tarefa_ID": "TA02",
    "NFR_Tipo": "Usability",
    "NFR_Atributo": "user interface",
    "NFR_Sentença": "The interface for editing access permissions should be intuitive and easy to navigate for administrators."
  },
  {
    "Tarefa_ID": "TA03",
    "NFR_Tipo": "Performance",
    "NFR_Atributo": "response time",
    "NFR_Sentença": "The system must respond to permission editing actions within 2 seconds under normal load conditions."
  },
  {
    "Tarefa_ID": "TA04",
    "NFR_Tipo": "Reliability",
    "NFR_Atributo": "availability",
    "NFR_Sentença": "The system must achieve 99.9% uptime to 